### Load data from bronze layer to silver Layer
#### Steps
1. Run the configurations file which has the folder paths defined.
2. Load into silver layer after casting datetime columns to date columns and dropping the rowguid column from all tables.

In [0]:
%run "../SetUp/Configuration"

In [0]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col, lit, date_format , from_utc_timestamp

In [0]:
# Populate the table names from bronze layer in a list 
table_names=[]
for table in dbutils.fs.ls(f'{bronze_folder_path}/SalesLT/'):
    table_names.append(table.name.split('/')[0])

for table in table_names:
    df = spark.read \
    .option("header", True)\
    .parquet(f'{bronze_folder_path}/SalesLT/{table}/{table}.parquet')
    
    # convert the datetime columns to date columns
    # Dropping the rowguid column from all tables
    for col in df.columns:
        if "Date" in col or "date" in col :
            df= df.withColumn(col,date_format(from_utc_timestamp(df[col].
            cast(TimestampType()),"UTC"),"yyyy-MM-dd"))
        
        if col == 'rowguid' :
            df = df.drop(col)
            
    # write the data to silver layer in delta format
    output_path = f'{silver_folder_path}/SalesLT/{table}/'
    df.write.format('delta').mode("overwrite").save(output_path)        